**Machine Learning Basic Principles 2018 - Data Analysis Project Report**

# *Music genre classification*

## Abstract

In this project our intent is to formulate an implementation of music classification to genres using machine learning techniques. A standard procedure before analyzing audio signals is decomposing the audio. Before the classification, the classifiable song needs to be preprocessed by extracting some distinct characteristics. The songs are decomposed to three different categories - rhythm patterns, chroma and mel frequency cepstral coefficients (MFCCs), which make up the feature space of the model. After the initial feature mapping, some additional features are computed that include variance, maximum, minimum and mean values of the three categories and added to the feature vector. Using these features the classification is implemented with C-support vector classification (SVC), which is first trained with provided labeled data and later used to predict labels of unlabeled data, that is,  predicting which of the 10 music genres considered in the project the songs belong to. The final fitted SVC classifier yielded prediction accuracies of approximately 66%.

## 1. Introduction

Dividing music to genres is an usual way of categorizing music since it comes in all different shapes and sizes. Musical genres shares some charasteristic features which include for instance rhythm patterns, timbre and  spectral qualities. 

Nowadays with growing music streaming platforms with new music flooding in around the clock, a convenient way of organizing and managing music is by categorizing it by its genre. Nevertheless, as no one can keep up with the amount of new music rushing to the market and that is where a genre classifier software would come in handy. Categorizing music by its genre helps for example the service provider to offer more specific and personalized music to the users and makes it easier for the users to find the kind of music they like. However, classifying a song solely on its tonal and rhytmic qualities is a rather hard task to do since there is no definite boundaries between different genres of music. 

The charasteristics we use in this project for genre classification are rhythm patterns, chroma and MFCCs. Chroma values depict the melody and harmony of the song and consist of 12 classes, one for each semitone, and 4 statistics. The rhythm pattern values consist of how much frequency modulation on specific range of the spectrum is happening and is represented by 24 different bands and 7 statistics. MFCCs are a way of describing the timble of a audio sample. To obtain the MFCCs, the audio is fourier transformed and mapped to mel scale which is then again transformed using discrete cosine transform. The resulting amplitudes are the mel frequency cepstral coefficients. The first 12 MFCCs with 4 statistics are inlcuded in the feature vector. Combining all the features we end up with a 264 dimensional feature vector [1].

The goal is to map each feature vector to the correct corresponding label. We implement a model which takes a feature vector as input and returns a vector with its value correspong to probability of the input song belonging to  each genre. The label space consist of most common 10 genres which are numbered from one to ten:

1. Pop_Rock

2. Electronic

3. Rap

4. Jazz

5. Latin

6. RnB

7. International

8. Country

9. Reggae

10. Blues

In section 2 we overview the given dataset and preprocess the feature vectors to a more suitable format. The classifier and other methods used in this project are described and evaluated in section 3 and in section 4 we present the results.




*Background, problem statement, motivation, many references, description of
contents. Introduces the reader to the topic and the broad context within which your
research/project fits*
*- What do you hope to learn from the project?*
*- What question is being addressed?*
*- Why is this task important? (motivation)*
*Keep it short (half to 1 page).*



## 2. Data analysis

*Briefly describe data (class distribution, dimensionality) and how will it affect
classification. Visualize the data. Don’t focus too much on the meaning of the features,
unless you want to.*

*- Include histograms showing class distribution.*



In [ ]:
# Import libraries
import pandas as pd
import numpy as np
#...
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Load the data and cleanup

In [ ]:
#Analysis of the input data
# ...

## 3. Methods and experiments

*- Explain your whole approach (you can include a block diagram showing the steps in your process).* 

*- What methods/algorithms, why were the methods chosen. *

*- What evaluation methodology (cross CV, etc.).*



In [ ]:
# Trials with ML algorithms

## 4. Results

*Summarize the results of the experiments without discussing their implications.*

*- Include both performance measures (accuracy and LogLoss).*

*- How does it perform on kaggle compared to the train data.*

*- Include a confusion matrix.*



In [ ]:
#Confusion matrix ...

## 5. Discussion/Conclusions

*Interpret and explain your results *

*- Discuss the relevance of the performance measures (accuracy and LogLoss) for
imbalanced multiclass datasets. *

*- How the results relate to the literature. *

*- Suggestions for future research/improvement. *

*- Did the study answer your questions? *



## 6. References

[1] MLBP 2018 Data Analysis Project - Problem Description

## Appendix
*Any additional material needed to complete the report can be included here. For example, if you want to keep  additional source code, additional images or plots, mathematical derivations, etc. The content should be relevant to the report and should help explain or visualize something mentioned earlier. **You can remove the whole Appendix section if there is no need for it.** *